In [7]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
pdf_folder = "/content/drive/MyDrive/Colab Notebooks/pdf2"
text_folder = "/content/drive/MyDrive/txt1"


In [ ]:
!pip install pymupdf


In [ ]:
import os
import fitz  # PyMuPDF

# Folder containing PDFs
pdf_folder = "/content/drive/MyDrive/Colab Notebooks/pdf2"
# Folder to save text files
text_folder = "texts"

# Create output folder if it doesn't exist
os.makedirs(text_folder, exist_ok=True)

# Loop over all PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        text_path = os.path.join(text_folder, filename.replace(".pdf", ".txt"))

        # Open and extract text
        with fitz.open(pdf_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()

        # Write text to file
        with open(text_path, "w", encoding="utf-8") as f:
            f.write(text)

print(" Conversion completed. Text files are saved in 'texts' folder.")


In [ ]:
import os

text_folder = "texts"
text_files = [os.path.join(text_folder, f) for f in os.listdir(text_folder) if f.endswith(".txt")]

all_text = ""
for file_path in text_files:
    with open(file_path, "r", encoding="utf-8") as f:
        all_text += f.read()

print(all_text[:500])

In [ ]:
chunk_size = 1000
text_chunks = [all_text[i:i+chunk_size] for i in range(0, len(all_text), chunk_size)]

print(f"Total number of chunks: {len(text_chunks)}")
print("First chunk:")
print(text_chunks[0])

## Choose and load an llm model

### Subtask:
Select a suitable LLM model from Hugging Face that can be fine-tuned for summarization and text generation. Load the model and its tokenizer.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3-8b"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"Model '{model_name}' and tokenizer loaded successfully.")

## Fine-tune the llm model

### Subtask:
Train the chosen LLM model on the prepared text data. This step will involve defining the training parameters, setting up the training loop, and monitoring the training progress.


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset

# Add a padding token if the tokenizer doesn't have one
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the text chunks
tokenized_chunks = tokenizer(text_chunks, return_tensors="pt", padding=True, truncation=True)

# Create a Hugging Face Dataset
dataset = Dataset.from_dict({
    "input_ids": tokenized_chunks["input_ids"],
    "attention_mask": tokenized_chunks["attention_mask"],
})

# Create a data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

print("Text data tokenized and dataset created.")

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    num_train_epochs=10,  # Number of training epochs
    per_device_train_batch_size=10,  # Batch size per device during training
    save_steps=10_000,  # Save model every 10,000 steps
    save_total_limit=2,  # Only keep the last 2 checkpoints
    logging_dir="./logs",  # Directory for storing logs
    logging_steps=100, # Log every 200 steps
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

print("Training arguments defined and Trainer instantiated.")

In [ ]:
# Start training
trainer.train()

print("Training completed.")

In [ ]:
def generate_summary_and_abstract(prompt, model, tokenizer, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2, do_sample=True, temperature=0.7):
    """
    Generates a summary and abstract from a user prompt using a fine-tuned LLM.

    Args:
        prompt (str): The user prompt.
        model: The fine-tuned Hugging Face model.
        tokenizer: The corresponding tokenizer.
        max_length (int): The maximum length of the generated text.
        num_return_sequences (int): The number of sequences to generate.
        no_repeat_ngram_size (int): The size of ngrams that should not be repeated.
        do_sample (bool): Whether to use sampling for generation.
        temperature (float): Controls the randomness in sampling.

    Returns:
        str: The generated text containing the summary and abstract.
    """
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", max_length=max_length, truncation=True, padding=True)

    # Generate text
    output_sequences = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=no_repeat_ngram_size,
        do_sample=do_sample,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id # Use eos_token_id as pad_token_id
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    return generated_text


In [ ]:
import re

def extract_summary_abstract_and_references(generated_text, original_chunks):
    """
    Extracts summary and abstract from generated text and finds relevant references.

    Args:
        generated_text (str): The text generated by the LLM.
        original_chunks (list): A list of original text chunks.

    Returns:
        tuple: A tuple containing the extracted summary (str), abstract (str),
               and relevant references (list of str).
    """
    summary = ""
    abstract = ""
    references = []


    summary_match = re.search(r"Summary:\s*(.*?)(?:Abstract:|$)", generated_text, re.DOTALL | re.IGNORECASE)
    if summary_match:
        summary = summary_match.group(1).strip()

    abstract_match = re.search(r"Abstract:\s*(.*?)(?:Summary:|$)", generated_text, re.DOTALL | re.IGNORECASE)
    if abstract_match:
        abstract = abstract_match.group(1).strip()

    if not summary and not abstract:
        # Simple split, might need adjustment based on actual output format
        split_text = generated_text.split('\n\n', 1)
        abstract = split_text[0].strip()
        if len(split_text) > 1:
            summary = split_text[1].strip()
        else:
            summary = abstract 

    keywords = set()
    keywords.update(summary.split()[:10])
    keywords.update(abstract.split()[:10])


    relevant_chunks = []
    for i, chunk in enumerate(original_chunks):
        if any(re.search(r'\b' + re.escape(keyword) + r'\b', chunk, re.IGNORECASE) for keyword in keywords if len(keyword) > 3): # Check for keywords with length > 3
            relevant_chunks.append(f"Reference from chunk {i+1}: ...{chunk[:200]}...") # Include snippet and chunk number

    references = relevant_chunks

    return summary, abstract, references

def generate_summary_abstract_and_references_with_prompt(prompt, model, tokenizer, original_chunks, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2, do_sample=True, temperature=0.7):
    """
    Generates summary, abstract, and references for a given prompt.

    Args:
        prompt (str): The user prompt.
        model: The fine-tuned Hugging Face model.
        tokenizer: The corresponding tokenizer.
        original_chunks (list): A list of original text chunks.
        max_length (int): The maximum length of the generated text.
        num_return_sequences (int): The number of sequences to generate.
        no_repeat_ngram_size (int): The size of ngrams that should not be repeated.
        do_sample (bool): Whether to use sampling for generation.
        temperature (float): Controls the randomness in sampling.

    Returns:
        tuple: A tuple containing the extracted summary (str), abstract (str),
               and relevant references (list of str).
    """
    generated_text = generate_summary_and_abstract(
        prompt,
        model,
        tokenizer,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=no_repeat_ngram_size,
        do_sample=do_sample,
        temperature=temperature
    )

    summary, abstract, references = extract_summary_abstract_and_references(generated_text, original_chunks)

    return summary, abstract, references

# Example usage:
user_prompt = "Summarize the key findings about data analysis in the provided documents."
summary, abstract, references = generate_summary_abstract_and_references_with_prompt(user_prompt, model, tokenizer, text_chunks)

print("--- Generated Summary ---")
print(summary)
print("\n--- Generated Abstract ---")
print(abstract)
print("\n--- Relevant References ---")
if references:
    for ref in references:
        print(ref)
else:
    print("No relevant references found.")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Choose the same model used before
model_name = "gpt2"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"Model '{model_name}' and tokenizer re-loaded successfully.")

# Re-run the example usage from the previous cell
user_prompt = "Summarize the key findings about data analysis in the provided documents."
summary, abstract, references = generate_summary_abstract_and_references_with_prompt(user_prompt, model, tokenizer, text_chunks)

print("--- Generated Summary ---")
print(summary)
print("\n--- Generated Abstract ---")
print(abstract)
print("\n--- Relevant References ---")
if references:
    for ref in references:
        print(ref)
else:
    print("No relevant references found.")

In [ ]:
import os

text_folder = "texts"
text_files = [os.path.join(text_folder, f) for f in os.listdir(text_folder) if f.endswith(".txt")]

all_text = ""
for file_path in text_files:
    with open(file_path, "r", encoding="utf-8") as f:
        all_text += f.read()

# Re-create text chunks
chunk_size = 1000
text_chunks = [all_text[i:i+chunk_size] for i in range(0, len(all_text), chunk_size)]

print(f"Total number of chunks: {len(text_chunks)}")
print("Text chunks re-created.")

user_prompt = "Summarize the key findings about data analysis in the provided documents."
summary, abstract, references = generate_summary_abstract_and_references_with_prompt(user_prompt, model, tokenizer, text_chunks)

print("--- Generated Summary ---")
print(summary)
print("\n--- Generated Abstract ---")
print(abstract)
print("\n--- Relevant References ---")
if references:
    for ref in references:
        print(ref)
else:
    print("No relevant references found.")

In [ ]:
# Add a padding token to the tokenizer if it doesn't have one
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Padding token set for the tokenizer.")

# Re-run the example usage
user_prompt = "Summarize the key findings about data analysis in the provided documents."
summary, abstract, references = generate_summary_abstract_and_references_with_prompt(user_prompt, model, tokenizer, text_chunks)

print("--- Generated Summary ---")
print(summary)
print("\n--- Generated Abstract ---")
print(abstract)
print("\n--- Relevant References ---")
if references:
    for ref in references:
        print(ref)
else:
    print("No relevant references found.")

## Evaluate the model (optional)

### Subtask:
Evaluate the performance of the fine-tuned model on a separate dataset to assess the quality of the generated summaries and abstracts.


In [ ]:
# Step 1: Create a small evaluation dataset
# We will take a few chunks and manually create reference summaries/abstracts
evaluation_data = []

# Take the first 3 chunks as an example
for i in range(3):
    original_text = text_chunks[i]
    # Manually create a simple reference summary and abstract for each chunk
    # In a real scenario, this would require careful manual annotation
    reference_summary = f"This is a manually created summary for chunk {i+1}."
    reference_abstract = f"This is a manually created abstract for chunk {i+1}."

    evaluation_data.append({
        "original_text": original_text,
        "reference_summary": reference_summary,
        "reference_abstract": reference_abstract
    })

print(f"Created evaluation dataset with {len(evaluation_data)} entries.")

# Step 2: Generate summaries and abstracts using the fine-tuned model
model_outputs = []


for item in evaluation_data:
    prompt = "Summarize and abstract the following text:\n" + item["original_text"]
    generated_summary, generated_abstract, generated_references = generate_summary_abstract_and_references_with_prompt(
        prompt, model, tokenizer, text_chunks
    )
    model_outputs.append({
        "original_text": item["original_text"],
        "reference_summary": item["reference_summary"],
        "reference_abstract": item["reference_abstract"],
        "generated_summary": generated_summary,
        "generated_abstract": generated_abstract,
        "generated_references": generated_references
    })

print(f"Generated summaries and abstracts for {len(model_outputs)} entries.")


In [ ]:
# Step 3 & 4: Implement and utilize ROUGE evaluation metrics
# Install the rouge package if not already installed
!pip install rouge_score

from rouge_score import rouge_scorer

# Initialize the ROUGE scorer
# Use different metrics: rouge1, rouge2, and rougeL
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

# Calculate ROUGE scores for each generated summary/abstract pair
for output in model_outputs:
    reference_summary = output["reference_summary"]
    generated_summary = output["generated_summary"]
    reference_abstract = output["reference_abstract"]
    generated_abstract = output["generated_abstract"]

    # Calculate scores for summary
    summary_scores = scorer.score(reference_summary, generated_summary)

    # Calculate scores for abstract
    abstract_scores = scorer.score(reference_abstract, generated_abstract)

    rouge_scores.append({
        "summary_rouge1": summary_scores["rouge1"].fmeasure,
        "summary_rouge2": summary_scores["rouge2"].fmeasure,
        "summary_rougel": summary_scores["rougeL"].fmeasure,
        "abstract_rouge1": abstract_scores["rouge1"].fmeasure,
        "abstract_rouge2": abstract_scores["rouge2"].fmeasure,
        "abstract_rougel": abstract_scores["rougeL"].fmeasure,
    })

# Calculate average ROUGE scores
avg_summary_rouge1 = sum([score["summary_rouge1"] for score in rouge_scores]) / len(rouge_scores)
avg_summary_rouge2 = sum([score["summary_rouge2"] for score in rouge_scores]) / len(rouge_scores)
avg_summary_rougel = sum([score["summary_rougel"] for score in rouge_scores]) / len(rouge_scores)

avg_abstract_rouge1 = sum([score["abstract_rouge1"] for score in rouge_scores]) / len(rouge_scores)
avg_abstract_rouge2 = sum([score["abstract_rouge2"] for score in rouge_scores]) / len(rouge_scores)
avg_abstract_rougel = sum([score["abstract_rougel"] for score in rouge_scores]) / len(rouge_scores)

print("\n--- Average ROUGE Scores ---")
print(f"Average Summary ROUGE-1: {avg_summary_rouge1:.4f}")
print(f"Average Summary ROUGE-2: {avg_summary_rouge2:.4f}")
print(f"Average Summary ROUGE-L: {avg_summary_rougel:.4f}")
print(f"Average Abstract ROUGE-1: {avg_abstract_rouge1:.4f}")
print(f"Average Abstract ROUGE-2: {avg_abstract_rouge2:.4f}")
print(f"Average Abstract ROUGE-L: {avg_abstract_rougel:.4f}")

# Step 5: Qualitative evaluation (manual review)
print("\n--- Qualitative Evaluation (Sample Review) ---")
# Print a sample of generated vs. reference summaries/abstracts for manual review
for i, output in enumerate(model_outputs):
    print(f"\n--- Entry {i+1} ---")
    print("Reference Summary:")
    print(output['reference_summary'])
    print("Generated Summary:")
    print(output['generated_summary'])
    print("Reference Abstract:")
    print(output['reference_abstract'])
    print("Generated Abstract:")
    print(output['generated_abstract'])
    print("Generated References:")
    if output['generated_references']:
        for ref in output['generated_references']:
            print(ref)
    else:
        print("No relevant references found.")


# CODE AS ONE

In [ ]:
import os
import fitz  # PyMuPDF

# Folder containing PDFs
pdf_folder = "/content/drive/MyDrive/Colab Notebooks/pdf2"

# List to store extracted text with source information
extracted_data = []

# Loop over all PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)

        # Open and extract text page by page
        with fitz.open(pdf_path) as doc:
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                text = page.get_text()
                extracted_data.append({
                    "text": text,
                    "filename": filename,
                    "page_num": page_num + 1  # Page numbers are 1-based
                })

print(f"Extracted text from {len(extracted_data)} pages across all PDFs.")

# Now, create chunks from the extracted data, preserving source information
chunk_size = 1000
text_chunks_with_info = []
current_chunk_text = ""
current_chunk_start_info = None

for item in extracted_data:
    text = item["text"]
    filename = item["filename"]
    page_num = item["page_num"]

    # If this is the start of a new chunk accumulation, record the source info
    if not current_chunk_start_info:
        current_chunk_start_info = {"filename": filename, "start_page": page_num}

    current_chunk_text += text

    # If the current chunk text is long enough or this is the last item, create a chunk
    while len(current_chunk_text) >= chunk_size:
        text_chunks_with_info.append({
            "text": current_chunk_text[:chunk_size],
            "filename": current_chunk_start_info["filename"],
            "start_page": current_chunk_start_info["start_page"]
        })
        current_chunk_text = current_chunk_text[chunk_size:]
        # Update start page for the next potential chunk from the same page
        if len(current_chunk_text) > 0:
             current_chunk_start_info = {"filename": filename, "start_page": page_num}
        else:
             current_chunk_start_info = None # Reset if the current page is fully consumed


# Add any remaining text as the last chunk
if len(current_chunk_text) > 0:
     text_chunks_with_info.append({
            "text": current_chunk_text,
            "filename": current_chunk_start_info["filename"],
            "start_page": current_chunk_start_info["start_page"]
        })


print(f"Created {len(text_chunks_with_info)} text chunks with source information.")
print("First chunk with info:")
print(text_chunks_with_info[0])

In [ ]:
import re

def extract_summary_abstract_and_references(generated_text, text_chunks_with_info):
    """
    Extracts summary and abstract from generated text and finds relevant references
    with filename and page number information.

    Args:
        generated_text (str): The text generated by the LLM.
        text_chunks_with_info (list): A list of dictionaries, where each dictionary
                                     contains 'text', 'filename', and 'start_page'.

    Returns:
        tuple: A tuple containing the extracted summary (str), abstract (str),
               and relevant references (list of str).
    """
    summary = ""
    abstract = ""
    references = []

    # Simple approach: Look for patterns indicating summary and abstract
    # This will heavily depend on how the model was fine-tuned and the nature of the text
    summary_match = re.search(r"Summary:\s*(.*?)(?:Abstract:|$)", generated_text, re.DOTALL | re.IGNORECASE)
    if summary_match:
        summary = summary_match.group(1).strip()

    abstract_match = re.search(r"Abstract:\s*(.*?)(?:Summary:|$)", generated_text, re.DOTALL | re.IGNORECASE)
    if abstract_match:
        abstract = abstract_match.group(1).strip()

    # If specific patterns are not found, take the beginning of the text as summary/abstract
    if not summary and not abstract:
        # Simple split, might need adjustment based on actual output format
        split_text = generated_text.split('\n\n', 1)
        abstract = split_text[0].strip()
        if len(split_text) > 1:
            summary = split_text[1].strip()
        else:
            summary = abstract # Or handle as needed

    # Find relevant references by searching for keywords/phrases in original chunks
    keywords = set()
    # Use first few words of summary and abstract as keywords
    keywords.update(summary.split()[:10])
    keywords.update(abstract.split()[:10])

    relevant_chunks = []
    for chunk_info in text_chunks_with_info:
        chunk_text = chunk_info["text"]
        filename = chunk_info["filename"]
        start_page = chunk_info["start_page"]

        if any(re.search(r'\b' + re.escape(keyword) + r'\b', chunk_text, re.IGNORECASE) for keyword in keywords if len(keyword) > 3): # Check for keywords with length > 3
            # Limit the snippet length for better readability
            snippet = chunk_text[:500] + "..." if len(chunk_text) > 500 else chunk_text
            relevant_chunks.append(f"Reference from {filename} (page {start_page}): {snippet}")

    references = relevant_chunks

    return summary, abstract, references

def generate_summary_abstract_and_references_with_prompt(prompt, model, tokenizer, text_chunks_with_info, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2, do_sample=True, temperature=0.7):
    """
    Generates summary, abstract, and references for a given prompt.

    Args:
        prompt (str): The user prompt.
        model: The fine-tuned Hugging Face model.
        tokenizer: The corresponding tokenizer.
        text_chunks_with_info (list): A list of dictionaries, where each dictionary
                                     contains 'text', 'filename', and 'start_page'.
        max_length (int): The maximum length of the generated text.
        num_return_sequences (int): The number of sequences to generate.
        no_repeat_ngram_size (int): The size of ngrams that should not be repeated.
        do_sample (bool): Whether to use sampling for generation.
        temperature (float): Controls the randomness in sampling.

    Returns:
        tuple: A tuple containing the extracted summary (str), abstract (str),
               and relevant references (list of str).
    """
    generated_text = generate_summary_and_abstract(
        prompt,
        model,
        tokenizer,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=no_repeat_ngram_size,
        do_sample=do_sample,
        temperature=temperature
    )

    summary, abstract, references = extract_summary_abstract_and_references(generated_text, text_chunks_with_info)

    return summary, abstract, references

# Example usage:
user_prompt = "Summarize the key findings about data analysis in the provided documents."
# Call the updated function with text_chunks_with_info
summary, abstract, references = generate_summary_abstract_and_references_with_prompt(user_prompt, model, tokenizer, text_chunks_with_info)

print("--- Generated Summary ---")
print(summary)
print("\n--- Generated Abstract ---")
print(abstract)
print("\n--- Relevant References ---")
if references:
    for ref in references:
        print(ref)
else:
    print("No relevant references found.")

In [ ]:
# Re-run the example usage to verify the updated reference format
user_prompt = "Summarize the key findings about data analysis in the provided documents."
summary, abstract, references = generate_summary_abstract_and_references_with_prompt(user_prompt, model, tokenizer, text_chunks_with_info)

print("--- Generated Summary ---")
print(summary)
print("\n--- Generated Abstract ---")
print(abstract)
print("\n--- Relevant References ---")
if references:
    for ref in references:
        print(ref)
else:
    print("No relevant references found.")

## Summary:

### Data Analysis Key Findings

*   The initial text extraction process was successfully modified to store the source filename and page number for each extracted text chunk, resulting in 2140 chunks from 831 pages.
*   The `extract_summary_abstract_and_references` function was updated to utilize the stored source information, enabling it to generate references that include the filename and starting page number.
*   Testing confirmed that the updated reference extraction function correctly displays references with the filename and page number format.

### Insights or Next Steps

*   The current reference extraction method is based on simple keyword matching; consider implementing a more sophisticated approach, such as semantic similarity search, to find more relevant references.
*   The model's ability to generate accurate summaries, abstracts, and identify relevant references depends heavily on its training data and fine-tuning process, which were not detailed here. Future work could involve fine-tuning the model specifically for this task on a dataset of research papers and their corresponding summaries, abstracts, and reference sections.
